In [1]:
!pip install spacy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -u "/content/drive/MyDrive/Colab_Notebooks/s2v_reddit_2015_md.zip" -d "/content"

Archive:  /content/drive/MyDrive/Colab_Notebooks/s2v_reddit_2015_md.zip


In [4]:
!pip install sense2vec

In [5]:
!python -m spacy download en_core_web_md

2021-04-10 16:55:06.194899: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [6]:
!pip install nltk

In [7]:
!pip install pandas

In [8]:
import nltk
nltk.download('wordnet')
from nltk.corpus import  wordnet
from typing import List
import spacy
#en_core_web_md
import string
from nltk.corpus import wordnet
from spacy.lang.en.stop_words import STOP_WORDS
import random
import numpy as np
from itertools import chain

#packages to find similar words by wordembedding and sense2vec
from sense2vec import Sense2VecComponent 
from sense2vec import Sense2Vec #for standalone
import pandas as pd

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
 #!/usr/bin/python


#global tags

class QueryExpansion:

    def __init__(self, queries: pd.DataFrame, top_syns: int = 5, top_similar: int=2):
        self.df_queries = queries
        
        self.nlp = spacy.load("en_core_web_md")
        self.top_syns = top_syns

        self.sense = self.nlp.add_pipe("sense2vec").from_disk("s2v_reddit_2015_md/s2v_old")
        self.standalone_s2v = Sense2Vec().from_disk("./s2v_reddit_2015_md/s2v_old") #it is not dubplicated
        
        self.tags = ['CD','JJ','RB', 'NN', 'NNS','NNP','NNPS', 'VB']
        self.pos_tags = ['PROPN','VERB','NOUN','NUM']

        self.top_similar = top_similar

    # Query Expansion
    def expansion(self, relation: bool = False, synonyms: bool = False, sensevec: bool=False, embedded: bool=False):
        
        if relation:
          result = []
          result = [*result, *self.get_comparation_superlation_nouns_from_original_data()]
          self.df_queries = pd.concat([self.df_queries, pd.DataFrame(result, columns=['topic', 'query', 'tag'])])

        if synonyms:
          result = []
          result = [*result, *self.synonyms()]
          self.df_queries = pd.concat([self.df_queries, pd.DataFrame(result, columns=['topic', 'query', 'tag'])])
        
        if sensevec:
          result = []
          result = [*result, *self.similarwords_sensevec()]
          self.df_queries = pd.concat([self.df_queries, pd.DataFrame(result, columns=['topic', 'query', 'tag'])])
        if embedded:
          result = []
          result = [*result, *self.similarwords_wordembedding()]
          self.df_queries = pd.concat([self.df_queries, pd.DataFrame(result, columns=['topic', 'query', 'tag'])])

        return self.df_queries

    def similarwords_replace(self, query, similar_words):
        import copy
        expanded_queries=[]
        for word, similar_words in similar_words.items():
            for similar_word in similar_words:
                new_query=copy.deepcopy(query).replace(word, similar_word)
                expanded_queries.append(new_query)
        return expanded_queries

    def similarwords_sensevec(self):
        result=[]
        for query in list(self.df_queries['topic'].unique()):
            doc = self.nlp(query)
            expanded_queries=[]
            similar_words={}
            for token in doc:
                top_similar_words=[]
                if token.tag_ in self.tags or token.pos_ in self.pos_tags:
                    if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                        try:
                            for e in token._.s2v_most_similar(self.top_similar):
                                word = e[0][0].strip() #get only word from ((word, tag), proba)
                                if (word != token.text) and (self.nlp(word)[0].lemma_ != token.lemma_):
                                    top_similar_words.append(word)
                        except ValueError as err:
                            for ent in doc.ents:
                                if ent.text == token.text:
                                    try:
                                        for e in ent._.s2v_most_similar(self.top_similar):
                                            word = e[0][0].strip() #get only word from ((word, tag), proba)
                                            if (word != token.text) and (self.nlp(word)[0].lemma_ != token.lemma_):
                                                top_similar_words.append(word)

                                    except ValueError as err:
                                        
                                        query = token._.s2v_other_senses[0] #get first similar words by entity_tag
                                        for e in self.standalone_s2v.most_similar(query, n=self.top_similar):
                                            word = e[0].split("|")[0].strip() #get only word from (word|tag, proba)
                                            if (word != token.text) and (self.nlp(word)[0].lemma_ != token.lemma_):
                                                top_similar_words.append(word)
                        
                        similar_words[token.text]=list(set(top_similar_words))
            expanded_queries=self.similarwords_replace(query, similar_words)
            
            i = 1
            for new_query in expanded_queries:
              result.append([query,new_query, 'sensevec_'+str(i)])
              i = i +1
            #result.append(expanded_queries)
        return result

    def remove_punc(self, query: str):
        table = str.maketrans(dict.fromkeys(string.punctuation))
        title = query.translate(table)
        return str(title)

    def synonyms(self):
        result = []

        for query in list(self.df_queries['topic'].unique()):
            new_title = self.remove_punc(query)
            syn_pro_title = list()
            temp = new_title
            new_title = self.nlp(new_title)
            for token in new_title:
                syn_token = self.find_syns_word(token)
                syn_pro_title.extend(
                    [syn for syn in list(set(syn_token)) if
                     syn != str(token.text)])  # distinct and remove the same words
            # print(syn_pro_title)
            # synonyms_by_titles.writelines(" ".join(list(set(syn_pro_title))) + "\n")
            # ToDo temp(org) + syns or only syns?
            result.append([query, temp + " " + " ".join(list(set(syn_pro_title))), 'syns'])
        return result

    def find_syns_word(self, token: str):
        syn_token = []
        if (token.pos_ == "NOUN"):
            # ToDo automate wordnet install
            for synset in wordnet.synsets(token.lemma_):
                for lemma in synset.lemmas()[:self.top_syns]:  # top 5 synonyms
                    if "_" not in lemma.name():  # not include the words with _ ex: basketball_game
                        syn_token.append(lemma.name())
                    else:
                        for w in lemma.name().split("_"):
                            if w not in STOP_WORDS:
                                syn_token.append(
                                    w)  # add words with _ to two words ex. laptop_computer -> laptop and computer
        syn_token = [w for w in syn_token if w != " " and len(w) != 0]

        # ToDo remove random influence
        if len(syn_token) > 5:
            return random.sample(syn_token,k=5)  # after top 5 synonyms + splited words -> long titles -> reducing the syns random with 5
        else:
            return syn_token

    def get_comparation_superlation_nouns_from_original_data(self):
        result =[]

        for query in list(self.df_queries['topic'].unique()):
            nouns_as_string = []
            doc = self.nlp(query)
            annotations = ['CC', 'CD',
                           'JJ', 'JJR', 'JJS',
                           'RB', 'RBR', 'RBS',
                           'NN', 'NNS', 'NNP', 'NNPS',
                           'VB']
            for token in doc:
                if token.tag_ in annotations:
                    nouns_as_string.append(token.text)
            result.append([query, ' '.join(nouns_as_string), 'annotation'])
        return result
    def similarwords_wordembedding(self):
        result = []
        for query in list(self.df_queries['topic'].unique()):
            doc = self.nlp(query)
            #expanded queries
            expanded_queries=[]
            similar_words={}
            for token in doc:
                top_similar_words=[]
                if token.tag_ in self.tags or token.pos_ in self.pos_tags:
                    if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                        try:
                            word=token.lemma_
                            
                            try:
                              key = self.nlp.vocab.strings[word]
                              vector = self.nlp.vocab.vectors[key]
                              vector_asarray = np.asarray([vector])
                              ms = self.nlp.vocab.vectors.most_similar(vector_asarray, n=self.top_similar)
                              #print(ms)
                              similar_embedding = [self.nlp.vocab.strings[w] for w in ms[0][0]] #get only text from most_similar
                              #checking again if similar words are the same word
                              for word in similar_embedding:
                                  if (word != token.text) and (self.nlp(word)[0].lemma_ != token.lemma_):
                                      top_similar_words.append(self.nlp(word)[0].lemma_.lower())
                            except KeyError as err:
                              print(err)
                        except ValueError as err:
                            print(err)
                        
                        similar_words[token.text]=list(set(top_similar_words)) #only unique words
            
            #replace with similar words for new queries.
            expanded_queries = self.similarwords_replace(query, similar_words)
            
            i = 1
            for new_query in expanded_queries:
              result.append([query,new_query, 'embedded_'+str(i)])
              i = i +1
        return result



In [11]:
if __name__ == "__main__":
    
    
    topics = ['What is the difference between sex and love?', 
             'Which is better, a laptop or a desktop?', 
             'Which is better, Canon or Nikon?', 
             'What are the best dish detergents?', 
             'What are the best cities to live in?', 
             'What is the longest river in the U.S.?', 
             'Which is healthiest: coffee, green tea or black tea and why?', 
             'What are the advantages and disadvantages of PHP over Python and vice versa?', 
             'Why is Linux better than Windows?', 'How to sleep better?', 
             'Should I buy an LCD TV or a plasma TV?', 
             'Train or plane? Which is the better choice?',
             'What is the highest mountain on Earth?',
             'Should one prefer Chinese medicine or Western medicine?',
              'What are the best washing machine brands?', 
             'Should I buy or rent?', 'Do you prefer cats or dogs, and why?', 
             'What is the better way to grill outdoors: gas or charcoal?', 
             'Which is better, MAC or PC?', 'What is better: to use a brush or a sponge?',
             'Which is better, Linux or Microsoft?', 'Which is better, Pepsi or Coke?', 'What is better, Google search or Yahoo search?', 
             'Which one is better, Netflix or Blockbuster?', 'Which browser is better, Internet Explorer or Firefox?', 
             'Which is a better vehicle: BMW or Audi?', 'Which one is better, an electric stove or a gas stove?', 
             'What planes are best, Boeing or Airbus?', 'Which is better, Disneyland or Disney World?', 
             'Should I buy an Xbox or a PlayStation?', 'Which has more caffeine, coffee or tea?', 
             'Which is better, LED or LCD Reception Displays?', 'What is better: ASP or PHP?', 
             'What is better for the environment, a real or a fake Christmas tree?', 'Do you prefer tampons or pads?', 
             'What IDE is better for Java: NetBeans or Eclipse?', 'Is OpenGL better than Direct3D in terms of portability to different platforms?', 
             'What are the differences between MySQL and PostgreSQL in performance?', 'Is Java code more readable than code written in Scala?', 
             'Which operating system has better performance: Windows 7 or Windows 8?', 'Which smartphone has a better battery life: Xperia or iPhone?', 
             'Which four wheel truck is better: Ford or Toyota?', 'Should I prefer a Leica camera over Nikon for portrait photographs?',
             'Which company has a larger capitalization: Apple or Microsoft?', 'Which laptop has a better durability: HP or Dell?',
             'Which beverage has more calories per glass: beer or cider?', 'Is admission rate in Stanford higher than that of MIT?',
             'Is pasta healthier than pizza?', 'Which city is more expensive to live in: San Francisco or New York?', 
             'Whose salary is higher: basketball or soccer players?']
    
    df = pd.DataFrame(list(zip(topics, topics, len(topics)*['original'])), columns=['topic', 'query', 'tag'])
    print(df)

    pd.set_option('display.max_columns', 4)
    expansion = QueryExpansion(df)
    df_relation = expansion.expansion(relation=True, synonyms=True, sensevec=True, embedded=True)
    

                                                topic  ...       tag
0        What is the difference between sex and love?  ...  original
1             Which is better, a laptop or a desktop?  ...  original
2                    Which is better, Canon or Nikon?  ...  original
3                  What are the best dish detergents?  ...  original
4                What are the best cities to live in?  ...  original
5              What is the longest river in the U.S.?  ...  original
6   Which is healthiest: coffee, green tea or blac...  ...  original
7   What are the advantages and disadvantages of P...  ...  original
8                   Why is Linux better than Windows?  ...  original
9                                How to sleep better?  ...  original
10             Should I buy an LCD TV or a plasma TV?  ...  original
11        Train or plane? Which is the better choice?  ...  original
12             What is the highest mountain on Earth?  ...  original
13  Should one prefer Chinese medi

In [12]:
!python -m spacy validate

2021-04-10 16:58:59.531180: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.5) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_md   >=3.0.0,<3.1.0   3.0.0   ✔



In [13]:
for q in topics:
  print(q)
  print(df_relation[df_relation['topic']==q])
  print(100*"=")

What is the difference between sex and love?
                                          topic  \
0  What is the difference between sex and love?   
0  What is the difference between sex and love?   
0  What is the difference between sex and love?   
0  What is the difference between sex and love?   
1  What is the difference between sex and love?   
2  What is the difference between sex and love?   
3  What is the difference between sex and love?   
4  What is the difference between sex and love?   
0  What is the difference between sex and love?   

                                               query         tag  
0       What is the difference between sex and love?    original  
0                            difference sex and love  annotation  
0  What is the difference between sex and love de...        syns  
0  What is the actual difference between sex and ...  sensevec_1  
1  What is the real difference between sex and love?  sensevec_2  
2  What is the difference between actual s

In [20]:
#df=df.reset_index(drop=True)
df_relation[df_relation['topic']=='What are the best cities to live in?']

,topic,query,tag
4,What are the best cities to live in?,What are the best cities to live in?,original
4,What are the best cities to live in?,best cities live,annotation
4,What are the best cities to live in?,What are the best cities to live in urban city...,syns
16,What are the best cities to live in?,What are the best towns to live in?,sensevec_1
17,What are the best cities to live in?,What are the best small cities to live in?,sensevec_2
11,What are the best cities to live in?,What are the best metropolis to live in?,embedded_1
